In [1]:
import os
import glob
# import math
import pickle
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scipy.linalg import pinv

In [2]:
from scipy.stats import pearsonr 
from statsmodels.stats.multitest import fdrcorrection

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import matplotlib.ticker as ticker

In [5]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

## Setup

In [19]:
class Config:
    def __init__(self):
        self.n_components = 5
        self.seed = 42 # np.random.randint(0, 10000)
        self.path_ver = ["local", "server"][0]

        if self.path_ver == "local":
            self.hsk_data_dir = os.path.join("..", "..", "my_Haskins_project", "Data", "Chang_et_al") 
        else: # server
            self.hsk_data_dir = os.path.join("..", "..", "Haskins_project", "Data_single_characters")
        
        self.master_path = os.path.join(self.hsk_data_dir, "Chang_2020_z_CSR.xlsx") 
            ## made through "Modify_data_Chang_for_CSR.R", originated from "AoA_character_naming.rds"
            ## 1,353 characters; columns = ["Char", "subject_id", "RT", "LogCF", "NS", "CON", "PC", "SC", "SAR", "IMG", "AoA"]

        self.csr_data_dir = os.path.join("Data_Linguistic", "Chang_et_al", "Naming")
        self.pca_data_dir = os.path.join(self.csr_data_dir, f"PCA_nc={self.n_components}_seed={self.seed}")
        os.makedirs(self.pca_data_dir, exist_ok=True)

        self.pca_model_path = os.path.join(self.pca_data_dir, "model.pkl")
        self.pc_loadings_path = os.path.join(self.pca_data_dir, "pc_loadings.xlsx")
        self.explained_vars_path = os.path.join(self.pca_data_dir, "pc_explained_vars.xlsx")
        self.char_data_path = os.path.join(self.pca_data_dir, "char_data.xlsx")
        self.subj_data_path_template = os.path.join(self.pca_data_dir, "subj_{}.xlsx")

        self.csr_result_dir = os.path.join("Output_Linguistic", "Chang_et_al", "Naming")
        os.makedirs(self.csr_result_dir, exist_ok=True)
        self.csr_result_path = os.path.join(self.csr_result_dir, f"PCA_nc={self.n_components}_seed={self.seed}_CSR_result.xlsx")
        self.csr_summary_path = self.csr_result_path.replace(".xlsx", "_summ.xlsx")
        self.csr_res_detail_path = os.path.join(self.csr_result_dir, f"PCA_nc={self.n_components}_seed={self.seed}_CSR_res_details.xlsx")

        self.coefs_lines_path_template = os.path.join("Figs", f"PCA_nc={self.n_components}_seed={self.seed}_coefs_{'{}'}.png")
        self.val_dist_path_template = os.path.join("Figs", f"PCA_nc={self.n_components}_seed={self.seed}_{'{}'}_distribution.png")

In [7]:
def fit_csr_function(df: pd.DataFrame):
    def _parse_df(df: pd.DataFrame):
        F_vals = df.iloc[:, :-1].values.astype(float) # feature values
        Y = df.iloc[:, -1].values.astype(float)
        nF = F_vals.shape[1] # number of features
        nT = len(Y) # number of trials
        nP = int((nF * 3 + nF ** 2 + 2) / 2) # number of parameters
        nI = sum(range(1, nF)) # number of interaction terms

        return  F_vals, Y, nF, nT, nP, nI

    def _construct_design_matrix(nT: int, nF: int, nI: int, F_vals: np.ndarray) -> np.ndarray:
        X = np.zeros((nT, 1 + nF + nF + nI))
        X[:, 0] = 1 # constant term
        X[:, 1:1 + nF] = F_vals # linear terms
        X[:, 1 + nF:1 + 2 * nF] = F_vals ** 2 # quadratic terms
        col = 1 + nF + nF
        for j in range(nF - 1):
            for k in range(j + 1, nF):
                X[:, col] = F_vals[:, j] * F_vals[:, k] # interaction terms
                col += 1

        return X
    
    def _evalute_model(Coefs: np.ndarray, X: np.ndarray, Y: np.ndarray, nT: int, nP: int) -> tuple:
        Y_pred = X @ Coefs
        residuals = Y - Y_pred
        RSS = np.sum(residuals ** 2)
        TSS = np.sum((Y - np.mean(Y)) ** 2)
        Rsq = 1 - RSS / TSS
        Rsq_adj = 1 - (RSS / (nT - nP - 1)) / (TSS / (nT - 1))
        # re_var = RSS / nT # residual error variance
        # log_lik = -0.5 * nT * (np.log(2 * math.pi * re_var) + 1) # log-likelihood
        # AIC = -2 * log_lik + 2 * nP
        # AICc = AIC + (2 * nP ** 2 + 2 * nP) / (nT - nP - 1)
        # BIC = -2 * log_lik + np.log(nT) * nP
        RMSE = np.sqrt(np.mean(residuals ** 2)) # root mean square error
        NRMSE = RMSE / (Y.max() - Y.min()) # normalized 

        return Y_pred, residuals, [Rsq, Rsq_adj, NRMSE]
    
    def _build_coef_names(feature_names: list, nF: int) -> list:
        coef_names = ["X0"] +  feature_names
        coef_names.extend([f"F{i+1}^2" for i in range(nF)])
        for j in range(nF - 1):
            for k in range(j + 1, nF):
                coef_names.append(f"F{j+1}F{k+1}")

        return coef_names

    kf = KFold(n_splits=5, shuffle=True, random_state=config.seed)
    result_list, details_list = [], []

    for fold, (train_index, test_index) in enumerate(kf.split(df)):
        df_train, df_test = df.iloc[train_index], df.iloc[test_index]

        ## Create design matrix and solve regression coefficients:
        F_vals_train, Y_train, nF, nT, nP, nI = _parse_df(df_train)
        X_train = _construct_design_matrix(nT, nF, nI, F_vals_train)
        Coefs = np.linalg.pinv(X_train) @ Y_train # pinv(X_train.T @ X_train) @ (X_train.T @ Y_train)
        Y_pred_train, re_train, model_evals_train = _evalute_model(Coefs, X_train, Y_train, nT, nP)
        train_details = pd.DataFrame({
            "Y": Y_train, 
            "Y_hat": Y_pred_train, 
            "residual": re_train
        })
        train_details.insert(0, "Type", "train")

        ## Evaluate on test set:
        F_vals_test, Y_test, _, nT_, _, _ = _parse_df(df_test)
        X_test = _construct_design_matrix(nT_, nF, nI, F_vals_test)
        Y_pred_test, re_test, model_evals_test = _evalute_model(Coefs, X_test, Y_test, nT_, nP)
        test_details = pd.DataFrame({
            "Y": Y_test, 
            "Y_hat": Y_pred_test, 
            "residual": re_test
        })
        test_details.insert(0, "Type", "test")
        
        ## Store results:
        results = {"Fold": fold + 1}
        coef_names = _build_coef_names(list(df.columns[:-1]), nF)
        results.update(dict(zip(coef_names, Coefs)))
        results.update({
            "Train_Rsq": model_evals_train[0],
            "Train_Rsq_adj": model_evals_train[1], 
            "Train_NRMSE": model_evals_train[-1], 
            "Test_Rsq": model_evals_test[0], 
            "Test_NRMSE": model_evals_test[-1]
        })
        results = pd.DataFrame(results, index=[0])
        result_list.append(results)

        ## 
        details = pd.concat([train_details, test_details], ignore_index=True)
        details.insert(0, "Fold", fold + 1)
        details_list.append(details)

    return (
        pd.concat(result_list, ignore_index=True), 
        pd.concat(details_list, ignore_index=True), 
    )

In [8]:
def fit_glm_function(df: pd.DataFrame):
    def _parse_df(df: pd.DataFrame):
        F_vals = df.iloc[:, :-1].values.astype(float) # feature values
        Y = df.iloc[:, -1].values.astype(float)
        nF = F_vals.shape[1] # number of features
        nT = len(Y) # number of trials
        nP = nF + 1
        
        return  F_vals, Y, nF, nT, nP 

    def _construct_design_matrix(nT: int, nF: int, F_vals: np.ndarray) -> np.ndarray:
        X = np.zeros((nT, 1 + nF))
        X[:, 0] = 1 # constant term
        X[:, 1:1 + nF] = F_vals # linear terms

        return X
    
    def _evalute_model(Coefs: np.ndarray, X: np.ndarray, Y: np.ndarray, nT: int, nP: int) -> tuple:
        Y_pred = X @ Coefs
        residuals = Y - Y_pred
        RSS = np.sum(residuals ** 2)
        TSS = np.sum((Y - np.mean(Y)) ** 2)
        Rsq = 1 - RSS / TSS
        Rsq_adj = 1 - (RSS / (nT - nP - 1)) / (TSS / (nT - 1))
        RMSE = np.sqrt(np.mean(residuals ** 2)) # root mean square error
        NRMSE = RMSE / (Y.max() - Y.min()) # normalized 

        return Y_pred, residuals, [Rsq, Rsq_adj, NRMSE]
    
    def _build_coef_names(feature_names: list, nF: int) -> list:
        return ["X0"] +  feature_names

    kf = KFold(n_splits=5, shuffle=True, random_state=config.seed)
    result_list, details_list = [], []

    for fold, (train_index, test_index) in enumerate(kf.split(df)):
        df_train, df_test = df.iloc[train_index], df.iloc[test_index]

        ## Create design matrix and solve regression coefficients:
        F_vals_train, Y_train, nF, nT, nP = _parse_df(df_train)
        X_train = _construct_design_matrix(nT, nF, F_vals_train)
        Coefs = np.linalg.pinv(X_train) @ Y_train # pinv(X_train.T @ X_train) @ (X_train.T @ Y_train)
        Y_pred_train, re_train, model_evals_train = _evalute_model(Coefs, X_train, Y_train, nT, nP)
        train_details = pd.DataFrame({
            "Y": Y_train, 
            "Y_hat": Y_pred_train, 
            "residual": re_train
        })
        train_details.insert(0, "Type", "train")

        ## Evaluate on test set:
        F_vals_test, Y_test, _, nT_, _ = _parse_df(df_test)
        X_test = _construct_design_matrix(nT_, nF, F_vals_test)
        Y_pred_test, re_test, model_evals_test = _evalute_model(Coefs, X_test, Y_test, nT_, nP)
        test_details = pd.DataFrame({
            "Y": Y_test, 
            "Y_hat": Y_pred_test, 
            "residual": re_test
        })
        test_details.insert(0, "Type", "test")
        
        ## Store results:
        results = {"Fold": fold + 1}
        coef_names = _build_coef_names(list(df.columns[:-1]), nF)
        results.update(dict(zip(coef_names, Coefs)))
        results.update({
            "Train_Rsq": model_evals_train[0],
            "Train_Rsq_adj": model_evals_train[1], 
            "Train_NRMSE": model_evals_train[-1], 
            "Test_Rsq": model_evals_test[0], 
            "Test_NRMSE": model_evals_test[-1]
        })
        results = pd.DataFrame(results, index=[0])
        result_list.append(results)

        ## 
        details = pd.concat([train_details, test_details], ignore_index=True)
        details.insert(0, "Fold", fold + 1)
        details_list.append(details)

    return (
        pd.concat(result_list, ignore_index=True), 
        pd.concat(details_list, ignore_index=True), 
    )

In [82]:
def plot_cormat(DF, fig_size=None):
    def _corr_p(x, y):
        mask = ~ (np.isnan(x) | np.isnan(y)) # only use rows without NaN in either column
        if np.std(x[mask]) == 0 or np.std(y[mask]) == 0:
            return np.nan # correlation is undefined if either variable has zero variance
        else:
            return pearsonr(x[mask], y[mask])[1]
            
    def _create_annot_mat(cormat, p_stacked, fdr=0.05):
        q_vals = fdrcorrection(p_stacked.dropna().values, alpha=fdr)[1]
        q_stacked = pd.Series(q_vals, index=p_stacked.index)
        q_mat = q_stacked.unstack()
        q_sig = q_mat.map(lambda x: "*" * sum( x <= t for t in [0.05, 0.01, 0.001] ))
        annot_mat = cormat.map(lambda x: f"{x:.2f}") + q_sig.reindex_like(cormat)
        return annot_mat.fillna("--")
    
    cormat = DF.corr()
    p_mat = DF.corr(method=lambda x, y: _corr_p(x, y))
    p_stacked = p_mat.where(np.tril(np.ones(p_mat.shape), k=-1).astype(bool)).stack()
    annot_mat = _create_annot_mat(cormat, p_stacked)
    
    cormat = cormat.iloc[1:, :-1]  
    annot_mat = annot_mat.iloc[1:, :-1]
    mask = np.zeros_like(cormat)
    mask[np.triu_indices_from(mask, k=1)] = True
    
    sns.set_theme(style='white', font_scale=2)
    plt.figure(figsize=fig_size, dpi=500)
    g = sns.heatmap(
        cormat, mask=mask, # square=True, 
        vmin=-1, vmax=1, linewidth=.5, cmap="RdBu_r", cbar=False, 
        annot=pd.DataFrame(annot_mat), fmt = ""
    )
    g.set_yticklabels(g.get_yticklabels(), rotation=0)
    plt.tight_layout()

In [10]:
config = Config()

In [11]:
overwrite = [True, False][0]

## Load data

In [21]:
master_df = pd.read_excel(config.master_path)
subj_list = list(master_df["subject_id"].unique())
char_df = master_df.loc[:, ["Char", "LogCF", "NS", "CON", "PC", "SC", "SAR", "IMG", "AoA"]].drop_duplicates()
char_df = char_df.set_index("Char")

In [22]:
print(char_df.shape)

(1353, 8)


In [83]:
#### Plot correlations among features
plot_cormat(char_df, fig_size=(10, 5.5))
plt.savefig(os.path.join("..", "figures", f"cormat ({char_df.shape[0]} chars).png"))
plt.close()

In [85]:
cols = char_df.columns[::-1]
vals = char_df.values.flatten()
bins = np.arange(min(vals), max(vals)+0.2, 0.2)
# bins = np.arange(-4, 4+0.2, 0.2)
sep_size = 150
y0 = sep_size * len(cols)
scale_size = 50
scl_x = 2.5
scl_y = y0 - sep_size + scale_size
# c_list = iter(sns.color_palette("Set2", 8))

fig, ax = plt.subplots(figsize=(8, 6), dpi=500)
for f in cols:
    h, edges = np.histogram(char_df[f].values, bins=bins)
    centers = 0.5 * (edges[:-1] + edges[1:])
    y0 -= sep_size
    ax.fill_between(centers, y0, y0 + h, step="mid", alpha=.6, linewidth=0) # , color=next(c_list)
    ax.plot(centers, y0 + h, drawstyle="steps-mid", color="white", alpha=.8)
    ax.axhline(y0, color="k", linewidth=1)
ax.plot([scl_x, scl_x], [scl_y, scl_y + scale_size], color="k", linewidth=1)
ax.plot([scl_x - .05, scl_x + .05], [scl_y, scl_y], color="k", linewidth=1)
ax.plot([scl_x - .05, scl_x + .05], [scl_y + scale_size, scl_y + scale_size], color="k", linewidth=1)
ax.text(scl_x + .15, scl_y + scale_size/2, f"{scale_size}", va="center", ha="left", fontsize=14)
ax.axvline(0, ymin=0, ymax=.9, color="k", ls="--", linewidth=1)
ax.set_yticks([i * sep_size for i in range(len(char_df.columns))])
ax.set_yticklabels(cols, fontsize=16)
ax.yaxis.set_major_locator(ticker.MultipleLocator(scale_size))
ax.grid(axis="y", alpha=0.2, ls="--")
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.grid(axis="x", alpha=0.2)
ax.set_axisbelow(True)
ax.tick_params(left=False)
ax.tick_params(axis="x", labelsize=16)
for s in ["top", "right", "left", "bottom"]:
    ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join("..", "figures", "data hist.png"))
plt.close()

## Perform PCA

In [ ]:
## Perform PCA and save the model 
pca = PCA(n_components=config.n_components, random_state=config.seed)
pca.fit(char_df)

with open(config.pca_model_path, "wb") as f:
    pickle.dump(pca, f)

In [ ]:
## Save loadings (https://www.jcchouinard.com/pca-loadings/)
eignvectors = pca.components_.T
eignvalues = pca.explained_variance_
loadings = pd.DataFrame(
    data=eignvectors * np.sqrt(eignvalues), 
    columns=[f"PC{i+1}" for i in range(config.n_components)], 
    index=char_df.columns
)
loadings.to_excel(config.pc_loadings_path)

In [ ]:
## Save explained variance
explained = pd.DataFrame(
    {'Explained Variance': pca.explained_variance_, 
     'Explained Variance Ratio': pca.explained_variance_ratio_}, 
     index=[f"PC{i+1}" for i in range(config.n_components)]
)
explained.to_excel(config.explained_vars_path)

In [ ]:
# ## Plot culmulated explained variance
# sns.set_theme(style='white', font_scale=1.2)
# fig, ax = plt.subplots(figsize=(5, 3), dpi=200)
# plt.bar(explained.index, explained["Explained Variance Ratio"], color='k', alpha=.6)
# plt.plot(np.cumsum(explained["Explained Variance Ratio"]), color='b')
# plt.plot(explained.index, np.cumsum(explained["Explained Variance Ratio"]), 'o', color='b')
# plt.axhline(0.8, color='r', linestyle='--')
# ax.set_ylabel("explained variance")
# plt.tight_layout()
# plt.savefig(os.path.join("Figs", "PC_explained_variance.png"))
# plt.close()

## Save PCA-transformed individualized datasets

In [ ]:
## Reduce original data
reduced_char_df = pca.transform(char_df)
reduced_char_df = pd.DataFrame(
    data=reduced_char_df, 
    columns=[f"PC{i+1}" for i in range(config.n_components)], 
    index=char_df.index
)
reduced_char_df.reset_index(inplace=True)
reduced_char_df.rename(columns={"index": "Char"})
reduced_char_df.to_excel(config.char_data_path, index=False)

In [ ]:
## Save it for each participant
overwrite = False

if len(glob.glob(config.subj_data_path_template.format("*"))) == 0 or overwrite:
    updated_master = (
        master_df.loc[:, ["Char", "subject_id", "RT"]]
        .merge(reduced_char_df, on="Char", how="left")
    )
    for subj in subj_list:
        subj_df = updated_master[updated_master["subject_id"] == subj]
        df = subj_df.loc[:, [f"PC{i+1}" for i in range(config.n_components)] + ["RT"]]
        df.to_excel(config.subj_data_path_template.format(subj), index=False)

In [ ]:
# ## Save spiltted data for a specific participant
# subj = 5017
# subj_df = updated_master[updated_master["subject_id"] == subj]
# df = subj_df.loc[:, [f"PC{i+1}" for i in range(config.n_components)] + ["RT"]]
# kf = KFold(n_splits=5, shuffle=True, random_state=config.seed)

# for fold, (train_index, test_index) in enumerate(kf.split(df)):
#     df_train, df_test = df.iloc[train_index], df.iloc[test_index]
#     df_train.to_excel(config.subj_data_path_template.format(f"{subj}_{fold+1}-train"), index=False)
#     df_test.to_excel(config.subj_data_path_template.format(f"{subj}_{fold+1}-test"), index=False)

## Fit CSR function on PCA-transformed individualized datasets

In [ ]:
## Original psycholinguistic variables
use_orig_vars = [True, False][0]

if use_orig_vars:
    config.subj_data_path_template = os.path.join(config.csr_data_dir, "zscored_sub_{}.xlsx")

    for k, v in vars(config).items():
        if not config.pca_data_dir in str(v):
            if f"PCA_nc={config.n_components}" in str(v):
                setattr(config, k, v.replace(f"PCA_nc={config.n_components}", "Orig"))
else:
    config = Config()

In [ ]:
config.csr_result_path

In [ ]:
# if not os.path.exists(config.csr_result_path) or overwrite:
if True:
    df_path_list = glob.glob(config.subj_data_path_template.format("*"))
    output_list, out_detail_list = [], []
    
    for df_path in df_path_list:
        df = pd.read_excel(df_path)
        sid = os.path.basename(df_path).split("_")[1].split(".")[0]
        if use_orig_vars:
            sid = os.path.basename(df_path).split("_")[2].split(".")[0] # original psycholinguistic variables
        
        # results_df, details_df = fit_csr_function(df) # fit CSR function with 5-fold cross-validation
        results_df, details_df = fit_glm_function(df)
        
        results_df.insert(0, "SID", sid)
        output_list.append(results_df)
        
        details_df.insert(0, "SID", sid)
        out_detail_list.append(details_df)
    
    output_df = pd.concat(output_list, ignore_index=True)
    # output_df.to_excel(config.csr_result_path, index=False)
    output_df.to_excel(config.csr_result_path.replace("CSR", "GLM"), index=False)

    out_detail_df = pd.concat(out_detail_list, ignore_index=True)
    # out_detail_df.to_excel(config.csr_res_detail_path, index=False)
    out_detail_df.to_excel(config.csr_res_detail_path.replace("CSR", "GLM"), index=False)
else:
    output_df = pd.read_excel(config.csr_result_path)

In [ ]:
# output_df.head()

In [ ]:
# nF = config.n_components
df = pd.read_excel(glob.glob(config.subj_data_path_template.format("*"))[0])
nF = df.shape[1] - 1
nP = int((nF * 3 + nF ** 2 + 2) / 2)

In [ ]:
fw = 5
if nF > 5:
    fw = 10

In [ ]:
sns.set_theme(style='whitegrid', font_scale=1.2)
sns.set_style({'grid.linestyle': '--'})

In [ ]:
## Check coefficients: different folds for a specific participant
sid = 5017
fig_path = config.coefs_lines_path_template.format(f"sub-{sid}")

if not os.path.exists(fig_path) or overwrite:
    color_list = sns.color_palette('Set2')
    fig, ax = plt.subplots(figsize=(fw, 3), dpi=200)
    
    for i in range(5):
        dat = output_df.query(f"SID == {sid}")
        if dat.empty:
            dat = output_df.query(f"SID == '{sid}'")
        dat = dat.iloc[i, 2:2+nP]
        plt.plot(dat[1::], color=color_list[i], alpha=.6)
        plt.plot(dat[1::], 'o', color=color_list[i], alpha=.6)
        # ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        
    plt.tight_layout()
    plt.savefig(fig_path)
    plt.close()

In [ ]:
subj_list = output_df["SID"].unique()

In [ ]:
## Check coefficients: across parcticipants
fig_path = config.coefs_lines_path_template.format("all-subjs")

if not os.path.exists(fig_path) or overwrite:
    color_list = sns.color_palette('Set2', len(subj_list))
    fig, ax = plt.subplots(figsize=(fw, 4), dpi=200)
    
    for i, sid in enumerate(subj_list):
        dat = output_df.query(f"SID == {sid}")
        if dat.empty:
            dat = output_df.query(f"SID == '{sid}'")
        dat = dat.iloc[:, 2:2+nP].mean(axis=0)
        plt.plot(dat[1::], color=color_list[i], alpha=.6)
        plt.plot(dat[1::], 'o', color=color_list[i], alpha=.6)
        # ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        
    plt.tight_layout()
    plt.savefig(fig_path)
    plt.close()

In [ ]:
data_desc = (
    output_df
    .groupby("SID")
    .mean()
    .iloc[:, 1::]
    .describe()
    .astype("float")
    .map(lambda x: f"{x:.3f}")
    .loc[["min", "mean", "max", "std"], :]
    .T
)
# data_desc.to_excel(config.csr_summary_path)
data_desc.to_excel(config.csr_summary_path.replace("CSR", "GLM"))
# print(data_desc)

In [ ]:
## Plot distribution of a variable across participants
for targ_col, xmin in zip(["Train_Rsq", "Test_Rsq"], [0, -1]):
    fig_path = config.val_dist_path_template.format(targ_col.replace('_', '-'))
    
    if not os.path.exists(fig_path) or overwrite:
        fig, ax = plt.subplots(figsize=(5, 3), dpi=200)
        sns.histplot(
            data=output_df.groupby("SID").mean(), 
            x=targ_col, kde=True, bins=30
        )
        plt.axvline(output_df[targ_col].mean(), color="red", linestyle="-")
        plt.axvline(output_df[targ_col].median(), color="lightgreen", linestyle="-")
        plt.axvline(0, color="black", linestyle="--")
        ax.set_ylabel("")
        # ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
        # ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        plt.tight_layout() 
        plt.savefig(fig_path)
        plt.close()